In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [4]:
!pip install --quiet unstructured_client unstructured

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
keras-cv 0.8.2 requires keras-core, which is not installed.
keras-nlp 0.9.3 requires keras-core, which is not installed.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
google-cloud-bigquery 2.34.4 requires packaging<22.0dev,>=14.3, but you have packaging 24.0 which is incompatible.
jupyterlab 4.1.6 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.1.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
libpysal 4.9.2 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
momepy 0.7.0 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
osmnx 1.9.2 requires shapely>=2.0, but you have shapely 1.8.5.post1 which is incompatible.
spopt 0.6.0 requires shapely>=

In [96]:
!pip install --quiet chromadb

In [9]:
from unstructured_client import UnstructuredClient
from unstructured_client.models import shared
from unstructured_client.models.errors import SDKError

from unstructured.chunking.title import chunk_by_title
from unstructured.partition.html import partition_html
#from unstructured.partition.pptx import partition_pptx
from unstructured.staging.base import dict_to_elements

import chromadb

In [13]:
import json
s = UnstructuredClient(
    api_key_auth='<Enter API key>',
    server_url='https://api.unstructured.io/general/v0/general',
)

In [12]:
cons_GenAI = partition_html(filename="/kaggle/input/example-files/Generative AI_ The Next Consumer Platform _ Andreessen Horowitz.html")
econ_GenAI = partition_html(filename="/kaggle/input/example-files/The Economic Case for Generative AI.html")
pros_GenAI = partition_html(filename="/kaggle/input/example-files/The Future of Prosumer_ The Rise of AI Workflows .html")

In [14]:
filename = "/kaggle/input/pdf-example/attention-is-all-you-need.pdf"

with open(filename, "rb") as f:
    files=shared.Files(
        content=f.read(),
        file_name=filename,
    )

req = shared.PartitionParameters(
    files=files,
    strategy="hi_res",
    hi_res_model_name="yolox",
    pdf_infer_table_structure=True,
    skip_infer_table_types=[],
)

try:
    resp = s.general.partition(req)
    pdf_elements = dict_to_elements(resp.elements)
except SDKError as e:
    print(e)

In [15]:
#Remove the footers and references
footers = [item for item in pdf_elements if item.category=="Footer"]
reference = [item for item in pdf_elements if item.category=="Title" and item.text=="References"]
reference_id = reference[0].to_dict()['element_id']
reference_pg = reference[0].to_dict()['metadata']['page_number']
reference_items = [item for item in pdf_elements if item.metadata.parent_id == reference_id or item.metadata.page_number>reference_pg]
pdf_adj_elements = [item for item in pdf_elements if item not in footers+reference+reference_items]

In [17]:
#Chunk by title and add to ChromaDB
elements = chunk_by_title(pdf_adj_elements+cons_GenAI+econ_GenAI+pros_GenAI)

In [18]:
! pip install --quiet langchain_cohere langchain_experimental

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
keras-cv 0.8.2 requires keras-core, which is not installed.
keras-nlp 0.9.3 requires keras-core, which is not installed.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 15.0.2 which is incompatible.
google-cloud-bigquery 2.34.4 requires packaging<22.0dev,>=14.3, but you have packaging 23.2 which is incompatible.
jupyterlab 4.1.6 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.1.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
kfp 2.5

In [28]:
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document

documents=[]
for el in elements:
    metadata = el.metadata.to_dict()
    filtered_metadata = {}
    del metadata['languages']
    for key,value in metadata.items():
        if isinstance(value,list):
            continue
        else:
            filtered_metadata[key] = value
    filtered_metadata['source'] = filtered_metadata['filename']
    documents.append(Document(page_content = el.text,metadata = filtered_metadata))

In [29]:
documents[104]

Document(page_content='Early research has found that chatbots can be effective in treating mental health. Products like Woebot and Wysa have demonstrated clinically validated results and have received FDA designations for efficacy in treating conditions like postpartum depression, chronic pain, and anxiety. In a time when more than 25% of adults suffer from a diagnosable mental health condition and we’re experiencing a nationwide shortage of therapists, chatbots may be a feasible solution for non-acute cases.', metadata={'file_directory': '/kaggle/input/example-files', 'filename': 'Generative AI_ The Next Consumer Platform _ Andreessen Horowitz.html', 'filetype': 'text/html', 'last_modified': '2024-05-04T05:01:13', 'page_number': 1, 'orig_elements': 'eJy9VU1vGzcQ/SsDAb2J2g9Jleyb27RNDg2CwkAPqSGMyNldwlxyQXK1Vor89w65ll0bQRGgTS/7QQ5n3rwZvvn454IM9WTjQavFNSyumgpxUx8bdUSs9scS12ta415VO9qpfbNYwqKniAojsj0f74cOg/5E6hDpIR6ks5HdBd78uPgJvTmDp0DoZQdsCI0brYLYYQTJj6NjU/Yp0cKRgJqGZNQnSmvaQvSEUdsWEkQ00BG

In [21]:
# LLM
import os
os.environ['COHERE_API_KEY'] = '<Enter API key>'

In [88]:
from langchain_cohere import CohereEmbeddings
embd = CohereEmbeddings()
vectorstore = Chroma.from_documents(documents=documents,embedding=embd)
vector_retriever = vectorstore.as_retriever(
    search_type='similarity',
    search_kwargs = {"k":5}
)

In [89]:
#Set up a tool to access the vector data store
from langchain.tools.retriever import create_retriever_tool
vectorstore_search = create_retriever_tool(
    retriever=vector_retriever,
    name="GenAI_content_expert",
    description="Retrieve relevant information from a vectorstore that contains expert information on GenAI and its industry usage"
)

In [41]:
!pip install --quiet duckduckgo-search

In [90]:
#Set up a tool to access internet search results
from langchain_community.tools import DuckDuckGoSearchResults
internet_search = DuckDuckGoSearchResults()
internet_search.name = "internet_search"
internet_search.description = "Returns a list of relevant document snippets for a textual query retrieved from the internet."

from langchain_core.pydantic_v1 import BaseModel,Field
class DDGoSearchInput(BaseModel):
    query: str = Field(description="Query to search the internet with")
internet_search.args_schema = DDGoSearchInput

In [91]:
#Set up a tool to execute a python script
from langchain.agents import Tool
from langchain_experimental.utilities import PythonREPL

python_repl = PythonREPL()
python_tool = Tool(
    name="python_repl",
    description="Executes python code and returns the result. The code runs in a static sandbox without interactive mode, so print output or save output to a file.",
    func=python_repl.run,
)
python_tool.name = "python_interpreter"

from langchain_core.pydantic_v1 import BaseModel, Field
class ToolInput(BaseModel):
    code: str = Field(description="Python code to execute.")
python_tool.args_schema = ToolInput

In [60]:
!pip install --quiet --upgrade elevenlabs

In [92]:
#Set up a tool to convert text to speech
os.environ["ELEVEN_API_KEY"] = '<API KEY>'
from langchain_community.tools import ElevenLabsText2SpeechTool
tts = ElevenLabsText2SpeechTool()
#tts.name = "text_to_speech"
#tts.description = "Translates text into speech"
tts_tool = Tool(
    name="text_to_speech",
    description="Translates text into speech",
    func = tts.stream_speech
)

class TextInput(BaseModel):
    text_input: str = Field(description="Text to translate into speech")
tts_tool.args_schema = TextInput

In [33]:
from langchain.agents import AgentExecutor
from langchain_cohere.react_multi_hop.agent import create_cohere_react_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_cohere.chat_models import ChatCohere

In [93]:
#Create an agent using the Cohere ReAct framework with access to the relevant tools for executing planned multi-tool use design pattern
llm = ChatCohere(model='command-r-plus',temperature=0.3)

#Preamble
preamble = """
You are a learning companion who answers any questions that the user has on the field of Generative AI and generates the relevant results.
You are equipped with a vectorstore to search for the relevant information in crafting an understandable response.
You are also equipped with an internet search tool to look up the information if the vectorstore provides limited context.
If some numerical calculations or visualizations are required, you are equipped with the python interpreter for generating the results.
You also have a text to speech tool to generate speech output if the user explicity asks for it.
If the user asks for something not related to generative AI, please politely draw attention towards the objective of learning generative AI
"""

prompt = ChatPromptTemplate.from_template('{input}')

In [94]:
agent = create_cohere_react_agent(
    llm = llm,
    prompt=prompt,
    tools=[vectorstore_search,internet_search,python_tool,tts_tool]
)

agent_executor = AgentExecutor(agent=agent,tools=[vectorstore_search,internet_search,python_tool,tts_tool],verbose=True)

In [95]:
response = agent_executor.invoke({
    "input":"What was the very first LLM model designed and how long ago was it designed? Please only give the name and the number of years as <name of model>:<number of years>.Please translate the final answer to audio using the text to speech tool",
    "preamble":preamble
})

response['output']



> Entering new AgentExecutor chain...

I will search for the first LLM model ever designed and then calculate the number of years since it was designed. I will then use the text-to-speech tool to generate the audio output.
{'tool_name': 'GenAI_content_expert', 'parameters': {'query': 'first LLM model ever designed'}}
countless model variants in our original codebase and tensor2tensor. Llion also experimented with novel model variants, was responsible for our initial codebase, and efﬁcient inference and visualizations. Lukasz and Aidan spent countless long days designing various parts of and implementing tensor2tensor, replacing our earlier codebase, greatly improving results and massively accelerating our research.

countless model variants in our original codebase and tensor2tensor. Llion also experimented with novel model variants, was responsible for our initial codebase, and efﬁcient inference and visualizations. Lukasz and Aidan spent countless long days designing various parts 

'Eliza:62\n\n [Audio for the above text](https://storage.googleapis.com/cohere-assistant-data/text_to_speech_responses/eliza62.wav)'